# Checkers
I have decided to join the Board class and the Game class into one class. The way they have implemented the Board class makes it difficult to implement a checkers board. At the moment it just stores {coordinate, player}. In checkers we need to move players, delete players, promote players etc.. Of course we could modify it, but from an OOP point of view I think it is fine for the Checkers game to possess and control a board internally.

We start with two lists of pieces. The boolean next to the location (i,j) is the king status. If the piece is taken we replace its location with None. That is, the lists P[0] and P[1] will always have size 12.

In [75]:
# A standard game of checkers. 8x8 board.
class Checkers():
    def __init__(self):
        # initial positions
        W = [(x,y) for y in range(0,3) for x in range(y%2,8,2)]
        B = [(x,y) for y in range(5,8) for x in range(y%2,8,2)]
        B.append((3,3))
        B.append((7,3))
        self.P = {'W' : W, 'B' : B}
        # we record king status as a mapping from location to boolean
        W_King = {location : False for location in W}
        B_King = {location : False for location in B}
        self.Kings = {'W' : W_King,'B' : B_King}
        self.to_move = 'W'

    def remove_self_intersections(self, piece):
        # There are 4 possible moves.
        turn = self.to_move
        if(turn == 'W'):
            moves = []
            if(piece[1] < 7):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] + 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] + 1))
            if(self.Kings[turn][piece] and piece[1] > 0):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] - 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] - 1))
            available_moves = []
            for m in moves:
                if m in self.P['W']:
                    continue
                available_moves.append(m)
            return available_moves
            
        if(turn == 'B'):
            moves = []
            if(piece[1] > 0 ):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] - 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] - 1))
            if(self.Kings[turn][piece] and piece[1] < 0):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] + 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] + 1))
            available_moves = []
            for m in moves:
                if m in self.P['B']:
                    continue
                available_moves.append(m)
            return available_moves
    
    def generate_action(self, i, piece, moves):
        print(i, piece, moves)
        other_player = 'B' if self.to_move == 'W' else 'W'
        other_player_piece_locations = self.P[other_player]
        for move in moves:
            if move in other_player_piece_locations:
                # if we find a piece that we MIGHT be able
                # to capture, we need to make sure we can.
                print('!!')
    
    def actions(self):
        actions = []
        for i in range(12):
            turn = self.to_move
            piece = self.P[turn][i]
            # If the piece is taken, just continue.
            if piece == None: continue
            moves = self.remove_self_intersections(piece)
            # If there are no moves ,just continue.
            if(len(moves) == 0): continue
            # action is a tuple:
            # (i, (l,m), [taken_pieces], make_king)
            action = self.generate_action(i, piece, moves)
            actions.append(action)
            # loop through actions, if we find an action that
            # involves taking a piece we return that action as
            # a singleton.
            

In [76]:
c = Checkers()
c.actions()

8 (0, 2) [(1, 3)]
9 (2, 2) [(1, 3), (3, 3)]
!!
10 (4, 2) [(3, 3), (5, 3)]
!!
11 (6, 2) [(5, 3), (7, 3)]
!!
